# Alkiviadis Kariotis 241735 HW#2 
ITC6010A1 - NATURAL LANGUAGE PROCESSING - Spring Term 2023

## Trigram Language Model using the Reuters dataset.Evaluation of the model and reporting its perplexity for different cases of smoothing.

In [1]:
import math
import collections
import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams

class TrigramLanguageModel:

    def __init__(self):
        # Initialize the count dictionaries and total_words variable
        self.unigram_counts = collections.defaultdict(int)
        self.bigram_counts = collections.defaultdict(int)
        self.trigram_counts = collections.defaultdict(int)
        self.total_words = 0

    def train(self, corpus):
        # Training the language model on the given corpus
        for sentence in corpus:
            # Update the counts for unigrams, bigrams, and trigrams
            for unigram in sentence:
                self.unigram_counts[unigram] += 1
                self.total_words += 1
            for bigram in bigrams(sentence, pad_left=True, pad_right=True):
                self.bigram_counts[bigram] += 1
            for trigram in trigrams(sentence, pad_left=True, pad_right=True):
                self.trigram_counts[trigram] += 1

    def score(self, sentence):
        # Calculate the log probability score for a given sentence
        score = 0.0
        for trigram in trigrams(sentence, pad_left=True, pad_right=True):
            if self.bigram_counts[trigram[:2]] > 0:
                trigram_count = self.trigram_counts[trigram]
                if trigram_count > 0:
                    score += math.log(trigram_count)
                else:
                    score += math.log(1e-10)  # assign a very small probability for unseen trigrams
                score -= math.log(self.bigram_counts[trigram[:2]])
            else:
                unigram_count = self.unigram_counts[trigram[2]]
                if unigram_count > 0:
                    score += math.log(unigram_count + 1)
                else:
                    score += math.log(1e-10)  # assign a very small probability for unseen unigrams
                score -= math.log(self.total_words + len(self.unigram_counts))
        return score


    def perplexity(self, corpus):
        # Calculate the perplexity of the model on a given corpus
        total_log_prob = 0
        test_words = 0
        for sentence in corpus:
            total_log_prob += self.score(sentence)
            test_words += len(sentence)
        return math.exp(-total_log_prob / test_words)


def main():
    # Download necessary NLTK resources
    nltk.download('reuters')
    nltk.download('punkt')

    # Prepare train and test corpus from the Reuters dataset
    train_corpus = [nltk.word_tokenize(reuters.raw(fileid)) for fileid in reuters.fileids() if fileid.startswith('training/')]
    test_corpus = [nltk.word_tokenize(reuters.raw(fileid)) for fileid in reuters.fileids() if fileid.startswith('test/')]

    # Create an instance of TrigramLanguageModel
    model = TrigramLanguageModel()

    # Train the model on the training corpus
    model.train(train_corpus)

    # Calculate and print the perplexity of the model on training and test data
    print("Perplexity of the model on training data: ", model.perplexity(train_corpus))
    print("Perplexity of the model on test data: ", model.perplexity(test_corpus))


if __name__ == "__main__":
    main()

[nltk_data] Downloading package reuters to /Users/alkis/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alkis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Perplexity of the model on training data:  6.020840213995279
Perplexity of the model on test data:  132854.35510278275


## Spell checker using the Trigram Language Model and the Noisy Channel Approach. Proposing candidates for correction based on both Edit Distance and the probability given by the language model.

In [2]:
import math
import collections
import nltk
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from nltk.metrics.distance import edit_distance

class TrigramLanguageModel:

    def __init__(self):
        self.unigram_counts = collections.defaultdict(int)  # Counts of individual words (unigrams)
        self.bigram_counts = collections.defaultdict(int)  # Counts of consecutive word pairs (bigrams)
        self.trigram_counts = collections.defaultdict(int)  # Counts of three consecutive words (trigrams)
        self.total_words = 0  # Total number of words in the training corpus

    def train(self, corpus):
        for sentence in corpus:
            for unigram in sentence:
                self.unigram_counts[unigram] += 1
                self.total_words += 1
            for bigram in bigrams(sentence, pad_left=True, pad_right=True):
                self.bigram_counts[bigram] += 1
            for trigram in trigrams(sentence, pad_left=True, pad_right=True):
                self.trigram_counts[trigram] += 1

    def score(self, sentence):
        score = 0.0
        for trigram in trigrams(sentence, pad_left=True, pad_right=True):
            if self.bigram_counts[trigram[:2]] > 0:  # Check if the bigram exists in the training data
                score += math.log(self.trigram_counts[trigram] + 1)  # Add smoothed trigram count to the score
                score -= math.log(self.bigram_counts[trigram[:2]] + len(self.unigram_counts))  # Subtract smoothed bigram count
            else:
                score += math.log(self.unigram_counts[trigram[2]] + 1)  # If bigram not found, use unigram count as a fallback
                score -= math.log(self.total_words + len(self.unigram_counts))  # Subtract total word count
        return score

    def spell_check(self, sentence):
        corrected_sentence = []
        for word in sentence:
            if word not in self.unigram_counts:  # Check if the word is in the training data
                candidates = self.generate_candidates(word)  # Generate candidate corrections for the misspelled word
                if candidates:
                    corrected_word = max(candidates, key=lambda w: (self.score([w]), -edit_distance(word, w)))  # Choose the best correction based on score and edit distance
                    corrected_sentence.append(corrected_word)
                else:
                    corrected_sentence.append(word)  # If no candidates found, keep the original word
            else:
                corrected_sentence.append(word)  # If the word is in the training data, keep it as is
        return corrected_sentence

    def generate_candidates(self, word):
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [L + R[1:] for L, R in splits if R]  # Delete one letter
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]  # Swap adjacent letters
        replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]  # Replace a letter with another
        inserts    = [L + c + R for L, R in splits for c in letters]  # Insert a letter
        return set(deletes + transposes + replaces + inserts)  # Return a set of unique candidate corrections

def main():
    nltk.download('reuters')  # Download the Reuters corpus
    nltk.download('punkt')  # Download the Punkt tokenizer for tokenization

    corpus = [nltk.word_tokenize(reuters.raw(fileid)) for fileid in reuters.fileids() if fileid.startswith('training/')]  # Tokenize and extract sentences from the Reuters training corpus

    model = TrigramLanguageModel()  # Create an instance of the TrigramLanguageModel class
    model.train(corpus)  # Train the model using the training corpus

    test_sentence = ['This', 'is', 'a', 'tst', 'sentnce', '.']
    print('Original sentence: ', ' '.join(test_sentence))
    corrected_sentence = model.spell_check(test_sentence)  # Perform spell checking on the test sentence
    print('Corrected sentence: ', ' '.join(corrected_sentence))

if __name__ == "__main__":
    main()

[nltk_data] Downloading package reuters to /Users/alkis/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /Users/alkis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original sentence:  This is a tst sentnce .
Corrected sentence:  This is a test sentence .


## Combined both aforementioned procedures for the Gutenberg project corpus

In [3]:
import math
import collections
import nltk
from nltk import bigrams, trigrams
from nltk.metrics.distance import edit_distance
import requests

class TrigramLanguageModel:

    def __init__(self):
        self.unigram_counts = collections.defaultdict(int)  # Counts of individual words (unigrams)
        self.bigram_counts = collections.defaultdict(int)  # Counts of consecutive word pairs (bigrams)
        self.trigram_counts = collections.defaultdict(int)  # Counts of three consecutive words (trigrams)
        self.total_words = 0  # Total number of words in the training corpus

    def train(self, corpus):
        for sentence in corpus:
            for unigram in sentence:
                self.unigram_counts[unigram] += 1
                self.total_words += 1
            for bigram in bigrams(sentence, pad_left=True, pad_right=True):
                self.bigram_counts[bigram] += 1
            for trigram in trigrams(sentence, pad_left=True, pad_right=True):
                self.trigram_counts[trigram] += 1

    def score(self, sentence):
        score = 0.0
        for trigram in trigrams(sentence, pad_left=True, pad_right=True):
            if self.bigram_counts[trigram[:2]] > 0:  # Check if the bigram exists in the training data
                score += math.log(self.trigram_counts[trigram] + 1)  # Add smoothed trigram count to the score
                score -= math.log(self.bigram_counts[trigram[:2]] + len(self.unigram_counts))  # Subtract smoothed bigram count
            else:
                score += math.log(self.unigram_counts[trigram[2]] + 1)  # If bigram not found, use unigram count as a fallback
                score -= math.log(self.total_words + len(self.unigram_counts))  # Subtract total word count
        return score

    def spell_check(self, sentence):
        corrected_sentence = []
        for word in sentence:
            if word not in self.unigram_counts:  # Check if the word is in the training data
                candidates = self.generate_candidates(word)  # Generate candidate corrections for misspelled words
                if candidates:
                    corrected_word = max(candidates, key=lambda w: (self.score([w]), -edit_distance(word, w)))  # Choose the best correction based on score and edit distance
                    corrected_sentence.append(corrected_word)
                else:
                    corrected_sentence.append(word)  # If no candidates found, keep the original word
            else:
                corrected_sentence.append(word)  # If the word is in the training data, keep it as is
        return corrected_sentence

    def generate_candidates(self, word):
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes    = [L + R[1:] for L, R in splits if R]  # Delete one letter
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]  # Swap adjacent letters
        replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]  # Replace a letter with another
        inserts    = [L + c + R for L, R in splits for c in letters]  # Insert a letter
        return set(deletes + transposes + replaces + inserts)  # Return a set of unique candidate corrections
    
    def perplexity(self, corpus):
        total_log_prob = 0
        test_words = 0
        for sentence in corpus:
            total_log_prob += self.score(sentence)  # Accumulate the score of each sentence
            test_words += len(sentence)  # Count the number of words in the test corpus
        return math.exp(-total_log_prob / test_words)  # Calculate perplexity as the exponential of the average negative log probability

def main():
    nltk.download('punkt')  # Download the Punkt tokenizer for tokenization

    url = "http://norvig.com/big.txt"  # URL of the text corpus to be used
    response = requests.get(url)  # Send a GET request to retrieve the text corpus
    corpus = nltk.sent_tokenize(response.text)  # Tokenize the corpus into sentences
    corpus = [nltk.word_tokenize(sent) for sent in corpus]  # Tokenize each sentence into words
    
    # Split the corpus into training and test sets
    split_point = int(len(corpus) * 0.8)  # Split point for the training and test data
    train_corpus = corpus[:split_point]  # Training corpus
    test_corpus = corpus[split_point:]  # Test corpus

    model = TrigramLanguageModel()  # Create an instance of the TrigramLanguageModel class
    model.train(corpus)  # Train the model using the entire corpus
    
    print("Perplexity of the model on training data: ", model.perplexity(train_corpus))  # Calculate and print the perplexity on the training data
    print("Perplexity of the model on test data: ", model.perplexity(test_corpus))  # Calculate and print the perplexity on the test data

    test_sentence = ['This', 'is', 'a', 'tst', 'sentnce', '.']
    print('Original sentence: ', ' '.join(test_sentence))
    corrected_sentence = model.spell_check(test_sentence)  # Perform spell checking on the test sentence
    print('Corrected sentence: ', ' '.join(corrected_sentence))

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /Users/alkis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Perplexity of the model on training data:  14860.635906644056
Perplexity of the model on test data:  14461.73760696
Original sentence:  This is a tst sentnce .
Corrected sentence:  This is a tut sentence .
